Чтобы отключить режим отладки (его ещё называют «режим разработки»), в *settings.py* нужно установить значение `DEBUG=False`. После этого страницы с ошибками будут отображены совершенно в ином виде. 

При отключении режима отладки в проекте перестанет показываться статика — например картинки, привязанные к записям. Это нормально: в «боевом режиме» Django ожидает, что статику будет раздавать специально настроенный сервер.  

Также после отключения режима разработки сайт может вообще перестать работать на локальном компьютере, а на удалённом может вернуться ошибка 400: *Bad Request*. 

При этом в терминале может появиться такое сообщение:

```
CommandError: You must set settings.ALLOWED_HOSTS if DEBUG is False. 
```

Причина в том, что в *settings.py* в списке `ALLOWED_HOSTS` (в переводе — «разрешённые хосты») не указан адрес сервера, на котором размещён проект.

Проверьте этот список, в нём должны быть указаны все адреса, на которых может быть размещён проект. Для запуска на локальном компьютере в списке должны быть адреса *localhost* и *127.0.0.1*; их надо внести в список `ALLOWED_HOSTS`:


In [ ]:
# settings.py

DEBUG = False

ALLOWED_HOSTS = [
    'localhost',
    '127.0.0.1',
    # Когда проект будет опубликован и станет доступен для пользователей,
    # в этот список нужно будет добавить и адреса домена, где он будет размещён,
    # например 'acme.not' и 'www.acme.not'
] 

***
## Страницы ошибок

В Django есть предустановленные страницы ошибок (например, для ошибки 404 — «страница не найдена» или для 403 — «запрос отклонён»); однако выглядят эти страницы так, будто разработчику сайта не было до них дела. Нельзя оставлять их в таком виде: пользователи будут неприятно удивлены.

![alt text](https://pictures.s3.yandex.net/resources/218_1682602442.png)

Создадим собственные страницы ошибок; они относятся ко всему проекту, поэтому будет логично описать их view-функции и шаблоны в приложении core. Пока что такого приложения нет; создайте и зарегистрируйте его самостоятельно.

***
## Ошибка 404: страница не найдена

Подготовим view-функцию для страницы 404:

```py


In [ ]:
# core/views.py
from django.shortcuts import render

def page_not_found(request, exception):
    # Переменная exception содержит отладочную информацию; 
    # выводить её в шаблон пользовательской страницы 404 мы не станем.
    return render(request, 'core/404.html', status=404) 


Шаблон этой страницы будет таким:


In [ ]:
<!-- templates/core/404.html -->
{% extends "base.html" %}
{% block title %}Custom 404{% endblock %}

{% block content %}
  <h1>Custom 404</h1>
  <p>Страницы с адресом {{ request.path }} не существует</p>
  <a href="{% url 'pages:homepage' %}">Идите на главную</a>
{% endblock %} 


Django распознаёт ошибки и вызывает для их обработки заготовленные view-функции. Имена этих функций хранятся в специальных переменных — хендлерах (англ. handlers). 

Адрес view-функции с ошибкой 404 хранится в переменной `handler404` (по умолчанию это view-функция `django.views.defaults.page_not_found)`. 

Хендлер можно переопределить: передать в него имя собственной view-функции. Для этого достаточно добавить в головной url-файл проекта такую строчку:


In [ ]:
# urls.py
...
handler404 = 'core.views.page_not_found' 

В результате при ошибке 404 отработает view-функция `page_not_found()` и отобразится кастомная страница ошибки. А уж настроить эту страницу — дело фронтендера.

![alt text](https://pictures.s3.yandex.net/resources/219_1682602469.png)

Обработка прочих ошибок выполняется точно так же. В Django заготовлены переменные handler400, handler403 и несколько других, [в документации есть их перечень](https://docs.djangoproject.com/en/5.1/ref/urls/#handler400).

***
## 403: ошибка проверки CSRF, запрос отклонён

Если при отправке формы не был отправлен csrf-токен — Django вернёт ошибку 403, но страница этой ошибки кастомизируется отдельно. Шаблон и view-функция готовятся так же, как и для других страниц, но переопределяется не хендлер, а константа `CSRF_FAILURE_VIEW` в *settings.py*.

View-функция:


In [ ]:
# core/views.py
from django.shortcuts import render

def csrf_failure(request, reason=''):
    return render(request, 'core/403csrf.html', status=403) 

Шаблон:


In [ ]:
<!-- templates/core/403csrf.html -->
{% extends "base.html" %}
{% block title %}Ошибка CSRF. 403{% endblock %}

{% block content %}
  <h1>Ошибка CSRF. 403</h1>
{% endblock %} 

Имя view-функции, обрабатывающей ошибку 403, указывается в *settings.py*, в константе `CSRF_FAILURE_VIEW`:


In [ ]:
# settings.py

CSRF_FAILURE_VIEW = 'core.views.csrf_failure' 

Проверьте, что при отключении режима отладки страницы ошибок отображаются в том же дизайне, что и остальные страницы сайта. После этого верните проект в режим разработки:

In [ ]:
# settings.py

DEBUG = True 